# STEP1.

In [1]:
class Variable:
  def __init__(self, data):
    self.data = data  # 통상값
    self.grad = None  # 미분값(s6에서 역전파를 이용한 미분 구현을 위해 Variable class 확장)
    self.creator = None  # 인스턴스 변수 추가 (s7)
    
  def set_creator(self, func):  # creator를 설정할 수 있도록 메서드 추가 (s7)
    self.creator = func

  def backward(self):  # s7 backward 메서드 추가
    f = self.creator  # 1. 함수를 가져온다
    if f is not None:
      x = f.input  # 2. 함수의 입력을 가져온다
      x.grad = f.backward(self.grad)  # 3. 함수의 backward 메서드를 호출한다.
      x. backward()  # 하나 앞 변수의 backward 메서드를 호출한다(재귀).


In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [3]:
data = np.array(2.0)  # x는 데이터를 담은 상자(인스턴스)이기 때문에, 인스턴스 변수(x.data)가 아닌, 일반 변수 data에 새로운 값 할당해도 인스턴스(객체) 변수는 변경되지 않음.
print(x.data)

1.0


In [4]:
x.data = np.array(2.) # Variable 클래스를 상자로 이용
print(x.data)

2.0


In [5]:
import numpy as np
x = np.array(1)
x.ndim



0

In [6]:
x = np.array([1, 2, 3])
x.ndim

1

In [7]:
x = np.array([[1, 2, 3], 
             [4, 5, 6]])  # 2차원 배열, 대괄호 2개 임에 유의!
x.ndim

2

# STEP2.

In [8]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)  # 기존 y = x ** 2로 작성하였으나, Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능만을 구한하고, 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현. 즉 구체적인 계산은 forward 메서드에서 하기 위해 수정. 
    output = Variable(y)
    output.set_creator(self)  # 출력 변수에 창조자를 설정한다. -> 이 부분이 연결을 동적으로 만드는 기법의 핵심(DeZero의 동적 계산 그래프)
    self.input = input  # s6. 역전파 시 활용하기 위해 입력 변수 기억(보관)한다.
    self.output = output  # 출력도 저장한다.
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):  # s6. 역전파로 확장
    raise NotImplementedError()

In [9]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y
  
  def backward(self, gy): # 역전파 메서드 추가확장
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [10]:
x = Variable(np.array(10))
f = Square() # 기존 f = Function() 에서 square() 함수로 세분화한 것에 따른 수정.
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# STEP3.

In [11]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [12]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# STEP4.

In [13]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)


In [14]:
f = Square()
x = Variable(np.array(2.))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [15]:
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


# STEP6. 

# STEP7.

In [16]:
# s7에서 수정된 Variable을 이용하여 역전파가 자동으로 실행됨을 확인.

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
y.backward()
# b.grad = C.backward(y.grad)
# a.grad = B.backward(b.grad)
# x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256
